In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests

In [5]:
df = pd.DataFrame.from_csv('Bluetooth_Travel_Sensors_-_Individual_Address_Files__IAF_.csv')

In [6]:
riverside_df = df[df.reader_identifier.str.contains(r'ih_35_riverside')]
cesar_chavez_df = df[df.reader_identifier.str.contains(r'cesar_chavez_ih_35')]

In [10]:
bridge = riverside_df.merge(cesar_chavez_df, how='inner', on='device_address')

,host_read_time_x,field_device_read_time_x,reader_identifier_x,device_address,host_read_time_y,field_device_read_time_y,reader_identifier_y
0,07/18/2016 12:00:38 AM,07/17/2016 11:54:31 PM,ih_35_riverside,8a:85:6f:ee:c6,07/18/2016 12:12:44 AM,07/18/2016 12:06:36 AM,cesar_chavez_ih_35
1,07/18/2016 01:24:11 AM,07/18/2016 01:18:03 AM,ih_35_riverside,8a:85:6f:ee:c6,07/18/2016 12:12:44 AM,07/18/2016 12:06:36 AM,cesar_chavez_ih_35
2,07/18/2016 12:00:46 AM,07/17/2016 11:54:39 PM,ih_35_riverside,43:12:ce:4d:9a,07/18/2016 12:01:34 AM,07/17/2016 11:55:28 PM,cesar_chavez_ih_35
3,07/18/2016 12:00:47 AM,07/17/2016 11:54:40 PM,ih_35_riverside,c6:34:4f:2c:f4,07/18/2016 12:01:58 AM,07/17/2016 11:55:52 PM,cesar_chavez_ih_35
4,07/18/2016 12:00:52 AM,07/17/2016 11:54:45 PM,ih_35_riverside,15:06:d5:60:cb,07/18/2016 01:46:48 AM,07/18/2016 01:40:40 AM,cesar_chavez_ih_35
5,07/18/2016 12:00:52 AM,07/17/2016 11:54:45 PM,ih_35_riverside,15:06:d5:60:cb,07/18/2016 02:37:41 AM,07/18/2016 02:31:32 AM,cesar_chavez_ih_35
6,07/18/2016 12:00:52 AM,07/17/2016 11:54:45 PM,ih_35_riverside,15:06:d5:60:cb,07/18/2016 06:57:14 AM,07/18/2016 06:51:07 AM,cesar_chavez_ih_35
7,07/18/2016 12:00:52 AM,07/17/2016 11:54:45 PM,ih_35_riverside,15:06:d5:60:cb,07/18/2016 09:04:37 AM,07/18/2016 08:58:28 AM,cesar_chavez_ih_35
8,07/18/2016 01:41:42 AM,07/18/2016 01:35:34 AM,ih_35_riverside,15:06:d5:60:cb,07/18/2016 01:46:48 AM,07/18/2016 01:40:40 AM,cesar_chavez_ih_35
9,07/18/2016 01:41:42 AM,07/18/2016 01:35:34 AM,ih_35_riverside,15:06:d5:60:cb,07/18/2016 02:37:41 AM,07/18/2016 02:31:32 AM,cesar_chavez_ih_35
